# 1. Imports

In [18]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import seaborn as sns
from typing import Optional

import category_encoders
from category_encoders import WOEEncoder
from catboost import CatBoostClassifier
from optuna import Trial
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline

from utils.eda import summary

# 2. Notebook settings

In [19]:
import warnings
warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:,.2f}'.format

# 3. Data Load

In [20]:
test_df = pd.read_csv('test.csv')
train_df = pd.read_csv('train.csv')
submission_df = pd.read_csv('sample_submission.csv')

# 4. Data Cleaning 

In [21]:
cat_cols = [column for column in train_df.columns if train_df[column].dtype == 'object']
num_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()

num_cols.remove('id')
num_cols.remove('CustomerId')
num_cols.remove('IsActiveMember')
num_cols.remove('HasCrCard')
num_cols.remove('Exited')
cat_cols.remove('Surname')
cat_cols.append('IsActiveMember')
cat_cols.append('HasCrCard')

train_df['HasCrCard'] = train_df['HasCrCard'].astype(bool)
train_df['IsActiveMember'] = train_df['IsActiveMember'].astype(bool)
test_df['HasCrCard'] = test_df['HasCrCard'].astype(bool)
test_df['IsActiveMember'] = test_df['IsActiveMember'].astype(bool)
train_df.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.00,3,0.00,2,True,False,"181,449.97",0
1,1,15749177,Okwudiliolisa,627,France,Male,33.00,1,0.00,2,True,True,"49,503.50",0
2,2,15694510,Hsueh,678,France,Male,40.00,10,0.00,2,True,False,"184,866.69",0
3,3,15741417,Kao,581,France,Male,34.00,2,"148,882.54",1,True,True,"84,560.88",0
4,4,15766172,Chiemenam,716,Spain,Male,33.00,5,0.00,2,True,True,"15,068.83",0


# 5. Model Setup

In [39]:
def instantiate_woe_encoder(trial : Trial) -> WOEEncoder:
  params = {
    'sigma': trial.suggest_float('sigma', 0.001, 5),
    'regularization': trial.suggest_float('regularization', 0, 5),
    'randomized': trial.suggest_categorical('randomized', [True, False])
  }
  return WOEEncoder(**params)

def instantiate_robust_scaler(trial : Trial) -> RobustScaler:
  params = {
    'with_centering': trial.suggest_categorical(
      'with_centering', [True, False]
    ),
    'with_scaling': trial.suggest_categorical(
      'with_scaling', [True, False]
    )
  }
  return RobustScaler(**params)

def instantiate_catboost(trial : Trial) -> CatBoostClassifier:
  params = {
    'n_estimators': trial.suggest_int('n_estimators', 50, 200, 10),
    'max_depth': trial.suggest_int('max_depth', 2, 10),
    'random_state': 42
  }
  return CatBoostClassifier(**params)

def instantiate_catboost(trial : Trial) -> CatBoostClassifier:
  params = {
    'n_estimators': trial.suggest_int('n_estimators', 50, 200, 10),
    'max_depth': trial.suggest_int('max_depth', 2, 10),
    'random_state': 42
  }
  return CatBoostClassifier(**params)

def instantiate_catboost(trial : Trial) -> CatBoostClassifier:
  params = {
    'n_estimators': trial.suggest_int('n_estimators', 50, 200, 10),
    'max_depth': trial.suggest_int('max_depth', 2, 10),
    'random_state': 42
  }
  return CatBoostClassifier(**params)


def instantiate_numerical_pipeline(trial : Trial) -> Pipeline:
  pipeline = Pipeline([
    ('scaler', instantiate_robust_scaler(trial))
  ])
  return pipeline

def instantiate_categorical_pipeline(trial : Trial) -> Pipeline:
  pipeline = Pipeline([
    ('encoder', instantiate_woe_encoder(trial))
  ])
  return pipeline

In [27]:
from sklearn.compose import ColumnTransformer

def instantiate_processor(trial : Trial, numerical_columns : list[str], categorical_columns : list[str]) -> ColumnTransformer:
  
  numerical_pipeline = instantiate_numerical_pipeline(trial)
  categorical_pipeline = instantiate_categorical_pipeline(trial)
  
  processor = ColumnTransformer([
    ('numerical_pipeline', numerical_pipeline, numerical_columns),
    ('categorical_pipeline', categorical_pipeline, categorical_columns)
  ])
  
  return processor

def instantiate_model(trial : Trial, numerical_columns : list[str], categorical_columns : list[str]) -> Pipeline:
  
  processor = instantiate_processor(
    trial, numerical_columns, categorical_columns
  )
  extra_trees = instantiate_catboost(trial)
  
  model = Pipeline([
    ('processor', processor),
    ('extra_trees', extra_trees)
  ])
  
  return model

In [28]:
def objective(trial : Trial, X : DataFrame, y : np.ndarray | Series, numerical_columns : Optional[list[str]]=None, categorical_columns : Optional[list[str]]=None, random_state : int=42) -> float:
  if numerical_columns is None:
    numerical_columns = [
      *X.select_dtypes(exclude=['object', 'category']).columns
    ]
  
  if categorical_columns is None:
    categorical_columns = [
      *X.select_dtypes(include=['object', 'category']).columns
    ]
  
  model = instantiate_model(trial, numerical_columns, categorical_columns)
  
  kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
  roc_auc_scorer = make_scorer(roc_auc_score, needs_proba=True)
  scores = cross_val_score(model, X, y, scoring=roc_auc_scorer, cv=kf)
  
  return np.min([np.mean(scores), np.median([scores])])

# 6. Training

In [40]:
from optuna import create_study

study = create_study(study_name='optimization', direction='maximize')

study.optimize(lambda trial: objective(trial, train_df[cat_cols + num_cols], train_df['Exited']), n_trials=100)

[I 2024-01-11 17:41:24,420] A new study created in memory with name: optimization


Learning rate set to 0.399367
0:	learn: 0.5125836	total: 4.41ms	remaining: 790ms
1:	learn: 0.4324972	total: 9.23ms	remaining: 821ms
2:	learn: 0.3900253	total: 13.8ms	remaining: 814ms
3:	learn: 0.3675980	total: 17.5ms	remaining: 771ms
4:	learn: 0.3543321	total: 21.3ms	remaining: 746ms
5:	learn: 0.3474672	total: 25.1ms	remaining: 728ms
6:	learn: 0.3431720	total: 29.9ms	remaining: 738ms
7:	learn: 0.3410476	total: 33.4ms	remaining: 719ms
8:	learn: 0.3383233	total: 37.1ms	remaining: 706ms
9:	learn: 0.3371615	total: 41.5ms	remaining: 705ms
10:	learn: 0.3365119	total: 45.6ms	remaining: 700ms
11:	learn: 0.3359070	total: 49ms	remaining: 686ms
12:	learn: 0.3353752	total: 52.5ms	remaining: 674ms
13:	learn: 0.3346408	total: 56.7ms	remaining: 672ms
14:	learn: 0.3343831	total: 61ms	remaining: 671ms
15:	learn: 0.3337174	total: 65.2ms	remaining: 668ms
16:	learn: 0.3333877	total: 68.8ms	remaining: 659ms
17:	learn: 0.3329842	total: 72.3ms	remaining: 650ms
18:	learn: 0.3327350	total: 77ms	remaining: 652m

[I 2024-01-11 17:41:28,806] Trial 0 finished with value: 0.8768430275176903 and parameters: {'with_centering': True, 'with_scaling': False, 'sigma': 3.2140507061781665, 'regularization': 1.8124802212302393, 'randomized': True, 'n_estimators': 180, 'max_depth': 6}. Best is trial 0 with value: 0.8768430275176903.


157:	learn: 0.3153607	total: 600ms	remaining: 83.5ms
158:	learn: 0.3152811	total: 603ms	remaining: 79.7ms
159:	learn: 0.3151937	total: 608ms	remaining: 76ms
160:	learn: 0.3151071	total: 612ms	remaining: 72.2ms
161:	learn: 0.3150181	total: 616ms	remaining: 68.4ms
162:	learn: 0.3149622	total: 619ms	remaining: 64.6ms
163:	learn: 0.3148677	total: 624ms	remaining: 60.8ms
164:	learn: 0.3147591	total: 628ms	remaining: 57.1ms
165:	learn: 0.3147318	total: 632ms	remaining: 53.3ms
166:	learn: 0.3146496	total: 636ms	remaining: 49.5ms
167:	learn: 0.3145810	total: 640ms	remaining: 45.7ms
168:	learn: 0.3144232	total: 644ms	remaining: 41.9ms
169:	learn: 0.3143407	total: 648ms	remaining: 38.1ms
170:	learn: 0.3142871	total: 652ms	remaining: 34.3ms
171:	learn: 0.3141994	total: 656ms	remaining: 30.5ms
172:	learn: 0.3140938	total: 660ms	remaining: 26.7ms
173:	learn: 0.3139892	total: 663ms	remaining: 22.9ms
174:	learn: 0.3139237	total: 667ms	remaining: 19.1ms
175:	learn: 0.3138368	total: 671ms	remaining: 15

[I 2024-01-11 17:41:33,423] Trial 1 finished with value: 0.8756362000720518 and parameters: {'with_centering': True, 'with_scaling': True, 'sigma': 4.491531280734782, 'regularization': 4.89264905899418, 'randomized': True, 'n_estimators': 190, 'max_depth': 6}. Best is trial 0 with value: 0.8768430275176903.


...

146:	learn: 0.3155149	total: 396ms	remaining: 116ms
147:	learn: 0.3154916	total: 399ms	remaining: 113ms
148:	learn: 0.3154669	total: 402ms	remaining: 111ms
149:	learn: 0.3154532	total: 405ms	remaining: 108ms
150:	learn: 0.3154135	total: 408ms	remaining: 105ms
151:	learn: 0.3153681	total: 411ms	remaining: 103ms
152:	learn: 0.3153586	total: 413ms	remaining: 100ms
153:	learn: 0.3153462	total: 416ms	remaining: 97.3ms
154:	learn: 0.3153174	total: 419ms	remaining: 94.5ms
155:	learn: 0.3152891	total: 422ms	remaining: 91.9ms
156:	learn: 0.3152499	total: 424ms	remaining: 89.2ms
157:	learn: 0.3152300	total: 427ms	remaining: 86.5ms
158:	learn: 0.3151999	total: 430ms	remaining: 83.8ms
159:	learn: 0.3151833	total: 433ms	remaining: 81.1ms
160:	learn: 0.3151573	total: 436ms	remaining: 78.5ms
161:	learn: 0.3151361	total: 438ms	remaining: 75.8ms
162:	learn: 0.3151145	total: 441ms	remaining: 73.1ms
163:	learn: 0.3150935	total: 444ms	remaining: 70.3ms
164:	learn: 0.3150685	total: 446ms	remaining: 67.6ms


[I 2024-01-11 17:47:06,825] Trial 98 finished with value: 0.8759821458333381 and parameters: {'with_centering': True, 'with_scaling': True, 'sigma': 4.6017891540904445, 'regularization': 2.857151990551992, 'randomized': True, 'n_estimators': 190, 'max_depth': 2}. Best is trial 85 with value: 0.8893754730836191.


168:	learn: 0.3312723	total: 398ms	remaining: 49.4ms
169:	learn: 0.3312418	total: 400ms	remaining: 47.1ms
170:	learn: 0.3312279	total: 403ms	remaining: 44.8ms
171:	learn: 0.3312272	total: 406ms	remaining: 42.5ms
172:	learn: 0.3312220	total: 408ms	remaining: 40.1ms
173:	learn: 0.3312186	total: 411ms	remaining: 37.8ms
174:	learn: 0.3312167	total: 413ms	remaining: 35.4ms
175:	learn: 0.3312039	total: 415ms	remaining: 33ms
176:	learn: 0.3311964	total: 418ms	remaining: 30.7ms
177:	learn: 0.3311686	total: 421ms	remaining: 28.4ms
178:	learn: 0.3311463	total: 423ms	remaining: 26ms
179:	learn: 0.3311387	total: 425ms	remaining: 23.6ms
180:	learn: 0.3311129	total: 428ms	remaining: 21.3ms
181:	learn: 0.3310994	total: 430ms	remaining: 18.9ms
182:	learn: 0.3310877	total: 432ms	remaining: 16.5ms
183:	learn: 0.3310725	total: 435ms	remaining: 14.2ms
184:	learn: 0.3310591	total: 437ms	remaining: 11.8ms
185:	learn: 0.3310520	total: 440ms	remaining: 9.45ms
186:	learn: 0.3310376	total: 442ms	remaining: 7.09

[I 2024-01-11 17:47:10,680] Trial 99 finished with value: 0.8893754455973347 and parameters: {'with_centering': True, 'with_scaling': True, 'sigma': 4.037902572471951, 'regularization': 3.451046079815495, 'randomized': False, 'n_estimators': 190, 'max_depth': 3}. Best is trial 85 with value: 0.8893754730836191.


148:	learn: 0.3154669	total: 398ms	remaining: 110ms
149:	learn: 0.3154532	total: 402ms	remaining: 107ms
150:	learn: 0.3154135	total: 405ms	remaining: 105ms
151:	learn: 0.3153681	total: 408ms	remaining: 102ms
152:	learn: 0.3153586	total: 411ms	remaining: 99.3ms
153:	learn: 0.3153462	total: 413ms	remaining: 96.7ms
154:	learn: 0.3153174	total: 416ms	remaining: 93.9ms
155:	learn: 0.3152891	total: 419ms	remaining: 91.2ms
156:	learn: 0.3152499	total: 421ms	remaining: 88.6ms
157:	learn: 0.3152300	total: 424ms	remaining: 85.9ms
158:	learn: 0.3151999	total: 427ms	remaining: 83.3ms
159:	learn: 0.3151833	total: 430ms	remaining: 80.6ms
160:	learn: 0.3151573	total: 433ms	remaining: 77.9ms
161:	learn: 0.3151361	total: 435ms	remaining: 75.2ms
162:	learn: 0.3151145	total: 438ms	remaining: 72.5ms
163:	learn: 0.3150935	total: 440ms	remaining: 69.8ms
164:	learn: 0.3150685	total: 443ms	remaining: 67.2ms
165:	learn: 0.3150499	total: 446ms	remaining: 64.5ms
166:	learn: 0.3150355	total: 449ms	remaining: 61.8

# 7. Model evaluation

In [41]:
study.best_params
best_trial = study.best_trial

In [42]:
model = instantiate_model(best_trial, num_cols, cat_cols)
model.fit(train_df[cat_cols + num_cols], train_df['Exited'])

Learning rate set to 0.418043
0:	learn: 0.5181971	total: 4.74ms	remaining: 896ms
1:	learn: 0.4416791	total: 8.59ms	remaining: 807ms
2:	learn: 0.4021879	total: 12ms	remaining: 751ms
3:	learn: 0.3760197	total: 15.5ms	remaining: 720ms
4:	learn: 0.3610020	total: 18.3ms	remaining: 678ms
5:	learn: 0.3517069	total: 21.1ms	remaining: 648ms
6:	learn: 0.3458814	total: 25ms	remaining: 652ms
7:	learn: 0.3396173	total: 28.3ms	remaining: 644ms
8:	learn: 0.3354883	total: 31.1ms	remaining: 625ms
9:	learn: 0.3331924	total: 33.9ms	remaining: 611ms
10:	learn: 0.3316210	total: 37.2ms	remaining: 605ms
11:	learn: 0.3285502	total: 40.8ms	remaining: 605ms
12:	learn: 0.3273370	total: 44.6ms	remaining: 607ms
13:	learn: 0.3262551	total: 47.5ms	remaining: 598ms
14:	learn: 0.3256566	total: 50.7ms	remaining: 591ms
15:	learn: 0.3251455	total: 53.7ms	remaining: 584ms
16:	learn: 0.3246429	total: 57.7ms	remaining: 587ms
17:	learn: 0.3241113	total: 61.9ms	remaining: 592ms
18:	learn: 0.3235947	total: 64.7ms	remaining: 58

Pipeline(steps=[('processor',
                 ColumnTransformer(transformers=[('numerical_pipeline',
                                                  Pipeline(steps=[('scaler',
                                                                   RobustScaler(with_centering=False))]),
                                                  ['CreditScore', 'Age',
                                                   'Tenure', 'Balance',
                                                   'NumOfProducts',
                                                   'EstimatedSalary']),
                                                 ('categorical_pipeline',
                                                  Pipeline(steps=[('encoder',
                                                                   WOEEncoder(regularization=2.5043207731889074,
                                                                              sigma=4.2126028961207815))]),
                                                  ['Geography', 'Gender',
                                                   'IsActiveMember',
                                                   'HasCrCard'])])),
                ('extra_trees',
                 <catboost.core.CatBoostClassifier object at 0x283bcf650>)])

In [ ]:
# Check for underfitting/overfitting by analyzing splits
# folds = range(1, kf.get_n_splits() + 1)
# plt.plot(folds, rmse_train, 'o-', color='green', label='train')
# plt.plot(folds, rmse_test, 'o-', color='red', label='test')
# plt.legend()
# plt.grid()
# plt.xlabel('Number of fold')
# plt.ylabel('mean squared error')
# plt.show()

In [43]:
test_probabilities = model.predict_proba(test_df[cat_cols + num_cols])[:, 1]

In [44]:
sub = pd.DataFrame()
sub['id'] = test_df['id']
sub['Exited'] = test_probabilities
sub = sub.set_index('id')
sub.head()

,Exited
id,
165034,0.03
165035,0.83
165036,0.02
165037,0.22
165038,0.37


In [45]:
sub.to_csv('submission.csv')